## Learning SQL embeddings for cardinality and cost estimation

This is a notebook containing the code for the "Learning SQL embeddings for cardinality and cost estimation" BSc thesis at the National and Kapodistrian University of Athens.

# Data preparation

Download and decompress our datasets.

In [ ]:
%cd <the path to where you have the download_datasets.sh file>
!bash download_datasets.sh
!tar -xvzf joblight_train.tar.gz
!tar -xvzf jobm.tar.gz

Format all the data in such a way that they are usable by the models.

In [3]:
# necessary imports
import re
import pickle

# move to the correct directory
%cd ./joblight_train/joblight-train-all

# save the table sizes of the IMDB database
imdb_table_sizes = {"cast_info": 36366040, "comp_cast_type": 4, "company_type": 4, "complete_cast": 126655,
                    "info_type": 113, "kind_type": 7, "link_type": 18, "movie_companies": 2618265, "movie_info": 15076160,
                    "movie_info_idx": 1376832, "movie_keyword": 4513320, "movie_link": 28468, "role_type": 12, "title": 2514512}

# extract the necessary data for JOB-light
job_light = []    # queries
table_sizes = []  # table sizes
labels = []       # cardinalities
costs = []        # costs

def load_qrep(fn):
    assert ".pkl" in fn
    with open(fn, "rb") as f:
        query = pickle.load(f)

    return query

for i in range(43294):
  qfn = str(i)+".pkl"
  qrep = load_qrep(qfn)
  new_string = qrep['sql'].replace(' AS ', ' ')
  tables = re.search('FROM(.*)WHERE', new_string).group(1)
  counter = 0
  table_size = [0, 0, 0, 0]
  for word in imdb_table_sizes.keys():
    if word in tables:
      table_size[counter] = imdb_table_sizes[word]
      counter += 1
  job_light.append(new_string)
  table_sizes.append(table_size)
  labels.append(float(sum([cardinality['actual'] for cardinality in (subplan['cardinality'] for subplan in qrep['subset_graph']['nodes'][:])])))
  costs.append(float(sum([exec_time['actual'] for exec_time in (subplan['exec_time'] for subplan in qrep['subset_graph']['nodes'][:])])))

# move to the correct directory
%cd ../../jobm/all_jobm

# extract the necessary data for JOB
jobm = []         # queries
table_sizesm = [] # table sizes
labelsm = []      # cardinalities

for i in range(100):
  qfn = str(i+1)+".pkl"
  qrep = load_qrep(qfn)
  new_string = qrep['sql'].replace(' AS ', ' ')
  tables = re.search('FROM(.*)WHERE', new_string).group(1)
  counter = 0
  table_size = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  for word in imdb_table_sizes.keys():
    if word in tables:
      table_size[counter] = imdb_table_sizes[word]
      counter += 1
  jobm.append(new_string)
  table_sizesm.append(table_size)
  labelsm.append(float(sum([cardinality['actual'] for cardinality in (subplan['cardinality'] for subplan in qrep['subset_graph']['nodes'][:])])))


/content/drive/MyDrive/thesis/joblight_train/joblight-train-all
/content/drive/MyDrive/thesis/jobm/all_jobm


## Models

Install the necessary modules

In [ ]:
!pip install transformers

# Basic

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# obtaining input queries and labels (cardinality or cost)
input_queries = job_light
input_labels = labels #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# tokenize dataset
size = len(input_queries)

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries[:int(size*0.8)],
                            add_special_tokens=True,
                            padding='longest',
                            max_length=None,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

encoded_corpus_test = tokenizer(text=input_queries[int(size*0.8):],
                            add_special_tokens=True,
                            padding='longest',
                            max_length=None,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids_test = encoded_corpus_test['input_ids']
attention_mask_test = encoded_corpus_test['attention_mask']

# split train, validation and test sets

input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, batch_size):
    labels_tens = torch.tensor(labels)
    dataset = TensorDataset(inputs, masks, labels_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, batch_size)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):
    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                eval_output =  model(b_input_ids, b_input_mask)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

#With standardization

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# obtaining input queries and labels (cardinality or cost)
input_queries = <put desired query dataset here>
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# tokenize dataset
size = len(input_queries)

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries[:int(size*0.8)],
                            add_special_tokens=True,
                            padding='longest',
                            max_length=None,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

encoded_corpus_test = tokenizer(text=input_queries[int(size*0.8):],
                            add_special_tokens=True,
                            padding='longest',
                            max_length=None,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids_test = encoded_corpus_test['input_ids']
attention_mask_test = encoded_corpus_test['attention_mask']

# split train, validation and test sets

input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, batch_size):
    labels_tens = torch.tensor(labels)
    dataset = TensorDataset(inputs, masks, labels_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):
    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                eval_output =  model(b_input_ids, b_input_mask)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

#Standardization and preprocessing

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

# obtaining input queries and labels (cardinality or cost)
input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# tokenize dataset
size = len(input_queries)

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries[:int(size*0.8)],
                            add_special_tokens=True,
                            padding='longest',
                            max_length=None,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

encoded_corpus_test = tokenizer(text=input_queries[int(size*0.8):],
                            add_special_tokens=True,
                            padding='longest',
                            max_length=None,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids_test = encoded_corpus_test['input_ids']
attention_mask_test = encoded_corpus_test['attention_mask']

# split train, validation and test sets

input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, batch_size):
    labels_tens = torch.tensor(labels)
    dataset = TensorDataset(inputs, masks, labels_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):
    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                eval_output =  model(b_input_ids, b_input_mask)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

# Truncation

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

# obtaining input queries and labels (cardinality or cost)
input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
size = len(input_queries)

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries[:int(size*0.8)],
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

encoded_corpus_test = tokenizer(text=input_queries[int(size*0.8):],
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids_test = encoded_corpus_test['input_ids']
attention_mask_test = encoded_corpus_test['attention_mask']

# split train, validation and test sets
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, batch_size):
    labels_tens = torch.tensor(labels)
    dataset = TensorDataset(inputs, masks, labels_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):
    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                eval_output =  model(b_input_ids, b_input_mask)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

## Example

It can take a long time to run the model on the full dataset without significant computational resources. An example of its output with all the data is the following:

Epoch 1/50
-----
Training Loss: 0.6263792095476477

Validation Loss: 0.5056275129318237

R² Train: 0.3734703264157514

R² Validation: 0.4855691207777376

Q-error Train: 3.305453580852479

Q-error Validation: 3.2212924519511623

Epoch 2/50
-----
Training Loss: 0.4220415407911016

Validation Loss: 0.4496574401855469

R² Train: 0.5778525088854958

R² Validation: 0.5427571535166092

Q-error Train: 3.169342072568896

Q-error Validation: 2.7176938294752224

Epoch 3/50
-----
Training Loss: 0.3839462616944254

Validation Loss: 0.36389797925949097

R² Train: 0.6159775441705999

R² Validation: 0.6305513043639062

Q-error Train: 3.2330322042014576

Q-error Validation: 3.1446021134992956

Epoch 4/50
-----
Training Loss: 0.36595107361747714

Validation Loss: 0.35223135352134705

R² Train: 0.6339801306519363

R² Validation: 0.6425509751170011

Q-error Train: 2.5985835675678493

Q-error Validation: 2.002590108586524

Epoch 5/50
-----
Training Loss: 0.35313365140498565

Validation Loss: 0.351616770029068

R² Train: 0.6467960237482051

R² Validation: 0.6430236526281277

Q-error Train: 2.4418899832491476

Q-error Validation: 2.0721219609033517

Epoch 6/50
-----
Training Loss: 0.34719677209817423

Validation Loss: 0.3376225531101227

R² Train: 0.6527336546983005

R² Validation: 0.6568136728588471

Q-error Train: 2.451022208669985

Q-error Validation: 1.9337873201502385

Epoch 7/50
-----
Training Loss: 0.3302652337794821

Validation Loss: 0.3134097754955292

R² Train: 0.6696689056973306

R² Validation: 0.6815532628657763

Q-error Train: 2.8180485394905124

Q-error Validation: 2.448247365505978

Epoch 8/50
-----
Training Loss: 0.3203124474055074

Validation Loss: 0.2965673506259918

R² Train: 0.6796261704473632

R² Validation: 0.6991716550770177

Q-error Train: 2.471554392546991

Q-error Validation: 2.424967714712766

Epoch 9/50
-----
Training Loss: 0.30827594977881523

Validation Loss: 0.27276352047920227

R² Train: 0.6916648452912659

R² Validation: 0.7234265002329364

Q-error Train: 2.469976398412188

Q-error Validation: 3.2448873674124457

Epoch 10/50
-----
Training Loss: 0.29187094181629236

Validation Loss: 0.2533513903617859

R² Train: 0.7080649725790696

R² Validation: 0.7432932783311175

Q-error Train: 2.3000584370887167

Q-error Validation: 1.8474129286696954

Epoch 11/50
-----
Training Loss: 0.2709184946593246

Validation Loss: 0.25212234258651733

R² Train: 0.7290496653395091

R² Validation: 0.7448058287925963

Q-error Train: 2.4849044110244605

Q-error Validation: 2.378509979477508

Epoch 12/50
-----
Training Loss: 0.2536282257309982

Validation Loss: 0.24619947373867035

R² Train: 0.7463225243627698

R² Validation: 0.7506716822939337

Q-error Train: 2.169723929152888

Q-error Validation: 1.9040034145887412

Epoch 13/50
-----
Training Loss: 0.243370993328946

Validation Loss: 0.22245445847511292

R² Train: 0.7565825955331635

R² Validation: 0.7749502104843015

Q-error Train: 2.414969715068117

Q-error Validation: 1.8254809175770956

Epoch 14/50
-----
Training Loss: 0.23384952004723655

Validation Loss: 0.2023150473833084

R² Train: 0.7661062337462998

R² Validation: 0.7956783709125155

Q-error Train: 3.0248677334777523

Q-error Validation: 3.656915663930562

Epoch 15/50
-----
Training Loss: 0.2213311508995265

Validation Loss: 0.21470589935779572

R² Train: 0.7786274991226547

R² Validation: 0.7832230373433039

Q-error Train: 2.0874112375436655

Q-error Validation: 4.561651962539864

Epoch 16/50
-----
Training Loss: 0.2158280208413237

Validation Loss: 0.20222777128219604

R² Train: 0.7841293774685573

R² Validation: 0.795802311081996

Q-error Train: 2.1256507396741036

Q-error Validation: 2.9716248277469877

Epoch 17/50
-----
Training Loss: 0.21077621360240606

Validation Loss: 0.20366927981376648

R² Train: 0.7891971447411077

R² Validation: 0.7942142900146185

Q-error Train: 2.027032196453424

Q-error Validation: 2.1816229754438976

Epoch 18/50
-----
Training Loss: 0.19975997228634182

Validation Loss: 0.18474720418453217

R² Train: 0.8002006073163741

R² Validation: 0.813373991547143

Q-error Train: 2.0819505078696343

Q-error Validation: 1.9553764353484455

Epoch 19/50
-----
Training Loss: 0.19562459466382495

Validation Loss: 0.1998322606086731

R² Train: 0.8043368243517686

R² Validation: 0.7983205145080368

Q-error Train: 1.975451577415217

Q-error Validation: 2.681332593141885

Epoch 20/50
-----
Training Loss: 0.1936186439738485

Validation Loss: 0.19062547385692596

R² Train: 0.8063432195325804

R² Validation: 0.8074037730704271

Q-error Train: 2.0354150300759515

Q-error Validation: 1.8139886174991744

Epoch 21/50
-----
Training Loss: 0.18967999683078288

Validation Loss: 0.19355548918247223

R² Train: 0.8102884924396729

R² Validation: 0.8043107299574423

Q-error Train: 2.3548526556398603

Q-error Validation: 1.96988455829343

Epoch 22/50
-----
Training Loss: 0.18541605819218557

Validation Loss: 0.18919335305690765

R² Train: 0.8145394942226515

R² Validation: 0.8087330692763872

Q-error Train: 2.1490776616980494

Q-error Validation: 1.8157930175078152

Epoch 23/50
-----
Training Loss: 0.1795571636261758

Validation Loss: 0.19267940521240234

R² Train: 0.8204043709483474

R² Validation: 0.8052432629516297

Q-error Train: 2.2985047852906386

Q-error Validation: 2.0560954384725605

Validation loss has not improved in 5 epochs, stopping early.

MAE Test:  13346795.93650263

MDAE Test:  5532605.0

MSE Test:  728272338886797.1

MAPE Test:  0.543916324187827

MDAPE Test:  0.3755998390401983

R² Test:  0.7806363375028371

Q-error Test:  2.213671969249493

#With table sizes added

Method (a)

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

input_queries = [preprocess_query(query) for query in job_light]#<put desired query dataset here>]
input_labels = labels#<put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)
input_table_sizes = table_sizes#<put equivalent table sizes here>

# limiting the number of queries to be used in the model
input_queries = input_queries[:500]#[:number of queries to be used]
input_labels = input_labels[:500]#[:number of queries to be used]
input_table_sizes = input_table_sizes[:500]#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries,
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

size = len(input_queries)

# spliting data into train, validation and test sets

input_ids_test = input_ids[int(size*0.8):]
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_test = attention_mask[int(size*0.8):]
attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

table_sizes_test = input_table_sizes[int(size*0.8):]
table_sizes_val = input_table_sizes[int(size*0.6):int(size*0.8)]
table_sizes_train = input_table_sizes[:int(size*0.6)]

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize_simple(self, train, validation, test):
        train_stand = torch.tensor(train, dtype=torch.float)
        validation_stand = torch.tensor(validation, dtype=torch.float)
        test_stand = torch.tensor(test, dtype=torch.float)

        stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / stds
        validation_stand = validation_stand / stds
        test_stand = test_stand / stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)
table_sizes_train, table_sizes_val, table_sizes_test = standardizer.standardize_simple(table_sizes_train, table_sizes_val, table_sizes_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, table_sizes, batch_size):
    labels_tens = torch.tensor(labels)
    table_sizes_tens = torch.tensor(table_sizes, dtype=torch.float)
    dataset = TensorDataset(inputs, masks, labels_tens, table_sizes_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, table_sizes_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, table_sizes_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, table_sizes_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

import torch.nn as nn
from transformers import BertForSequenceClassification, BertConfig, BertModel

# load bert model within our defined network
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)


class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.linear2 = nn.Linear(D_out+4, D_out)  #+4 because we are adding table sizes
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks, table_sizes):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        embedding_with_sizes = torch.cat(tensors=[outputs, table_sizes], dim=1)
        outputs = self.linear2(embedding_with_sizes)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to gpu
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels, batch_table_sizes = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks, batch_table_sizes).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels, b_table_sizes = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask, b_table_sizes)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels, b_table_sizes = batch

                eval_output =  model(b_input_ids, b_input_mask, b_table_sizes)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for visualisation
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else 1e-10) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else 1e-10) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

Method (b)

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

input_queries = [preprocess_query(query) for query in job_light]#<put desired query dataset here>]
input_labels = labels#<put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)
input_table_sizes = table_sizes#<put equivalent table sizes here>

# limiting the number of queries to be used in the model
input_queries = input_queries[:500]#[:number of queries to be used]
input_labels = input_labels[:500]#[:number of queries to be used]
input_table_sizes = input_table_sizes[:500]#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries,
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

size = len(input_queries)

# spliting data into train, validation and test sets

input_ids_test = input_ids[int(size*0.8):]
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_test = attention_mask[int(size*0.8):]
attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

table_sizes_test = input_table_sizes[int(size*0.8):]
table_sizes_val = input_table_sizes[int(size*0.6):int(size*0.8)]
table_sizes_train = input_table_sizes[:int(size*0.6)]

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize_simple(self, train, validation, test):
        train_stand = torch.tensor(train, dtype=torch.float)
        validation_stand = torch.tensor(validation, dtype=torch.float)
        test_stand = torch.tensor(test, dtype=torch.float)

        stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / stds
        validation_stand = validation_stand / stds
        test_stand = test_stand / stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)
table_sizes_train, table_sizes_val, table_sizes_test = standardizer.standardize_simple(table_sizes_train, table_sizes_val, table_sizes_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, table_sizes, batch_size):
    labels_tens = torch.tensor(labels)
    table_sizes_tens = torch.tensor(table_sizes, dtype=torch.float)
    dataset = TensorDataset(inputs, masks, labels_tens, table_sizes_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, table_sizes_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, table_sizes_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, table_sizes_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

import torch.nn as nn
from transformers import BertForSequenceClassification, BertConfig, BertModel

# load bert model within our defined network
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)


class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.linear2 = nn.Linear(D_out+4, D_out+4)  #+4 because we are adding table sizes
        self.linear3 = nn.Linear(D_out+4, D_out+4)  #+4 because we are adding table sizes
        self.linear4 = nn.Linear(D_out+4, D_out)  #+4 because we are adding table sizes
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks, table_sizes):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        embedding_with_sizes = torch.cat(tensors=[outputs, table_sizes], dim=1)
        outputs = self.linear2(embedding_with_sizes)
        outputs = self.relu(outputs)
        outputs = self.linear3(embedding_with_sizes)
        outputs = self.relu(outputs)
        outputs = self.linear4(embedding_with_sizes)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to gpu
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels, batch_table_sizes = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks, batch_table_sizes).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels, b_table_sizes = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask, b_table_sizes)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels, b_table_sizes = batch

                eval_output =  model(b_input_ids, b_input_mask, b_table_sizes)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for visualisation
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else 1e-10) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else 1e-10) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

Method (c)

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)
input_table_sizes = put equivalent table sizes here>

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]
input_table_sizes = input_table_sizes#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries,
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

size = len(input_queries)

# spliting data into train, validation and test sets

input_ids_test = input_ids[int(size*0.8):]
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_test = attention_mask[int(size*0.8):]
attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

table_sizes_test = input_table_sizes[int(size*0.8):]
table_sizes_val = input_table_sizes[int(size*0.6):int(size*0.8)]
table_sizes_train = input_table_sizes[:int(size*0.6)]

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize_simple(self, train, validation, test):
        train_stand = torch.tensor(train, dtype=torch.float)
        validation_stand = torch.tensor(validation, dtype=torch.float)
        test_stand = torch.tensor(test, dtype=torch.float)

        stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / stds
        validation_stand = validation_stand / stds
        test_stand = test_stand / stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)
table_sizes_train, table_sizes_val, table_sizes_test = standardizer.standardize_simple(table_sizes_train, table_sizes_val, table_sizes_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, table_sizes, batch_size):
    labels_tens = torch.tensor(labels)
    table_sizes_tens = torch.tensor(table_sizes, dtype=torch.float)
    dataset = TensorDataset(inputs, masks, labels_tens, table_sizes_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, table_sizes_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, table_sizes_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, table_sizes_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

import torch.nn as nn
from transformers import BertForSequenceClassification, BertConfig, BertModel

# load bert model within our defined network
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in+4, D_out)  #+4 because we are adding table sizes
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks, table_sizes):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        embedding_with_sizes = torch.cat(tensors=[outputs, table_sizes], dim=1)
        outputs = self.linear(embedding_with_sizes)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to gpu
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels, batch_table_sizes = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks, batch_table_sizes).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels, b_table_sizes = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask, b_table_sizes)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels, b_table_sizes = batch

                eval_output =  model(b_input_ids, b_input_mask, b_table_sizes)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for visualisation
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else 1e-10) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else 1e-10) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

#Only table sizes

In [ ]:
# necessary imports
import re
import statistics
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)
input_table_sizes = <put equivalent table sizes here>

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]
input_table_sizes = input_table_sizes#[:number of queries to be used]

size = len(input_queries)

# spliting data into train, validation and test sets

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

table_sizes_test = input_table_sizes[int(size*0.8):]
table_sizes_val = input_table_sizes[int(size*0.6):int(size*0.8)]
table_sizes_train = input_table_sizes[:int(size*0.6)]

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize_simple(self, train, validation, test):
        train_stand = torch.tensor(train, dtype=torch.float)
        validation_stand = torch.tensor(validation, dtype=torch.float)
        test_stand = torch.tensor(test, dtype=torch.float)

        stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / stds
        validation_stand = validation_stand / stds
        test_stand = test_stand / stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)
table_sizes_train, table_sizes_val, table_sizes_test = standardizer.standardize_simple(table_sizes_train, table_sizes_val, table_sizes_test)

# make dataloaders
batch_size = 64

def make_dataloader(labels, table_sizes, batch_size):
    labels_tens = torch.tensor(labels)
    table_sizes_tens = torch.tensor(table_sizes, dtype=torch.float)
    dataset = TensorDataset(labels_tens, table_sizes_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(labels_train, table_sizes_train, batch_size)
valid_dataloader = make_dataloader(labels_val, table_sizes_val, batch_size)
test_dataloader = make_dataloader(labels_test, table_sizes_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

class Regressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(Regressor, self).__init__()
        D_in, D_out = 4, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_in)
        self.linear2 = nn.Linear(D_in, D_in)
        self.linear3 = nn.Linear(D_in, D_in)
        self.linear4 = nn.Linear(D_in, D_out)
        self.relu = nn.ReLU()
    def forward(self, table_sizes):
        outputs = self.linear(table_sizes)
        outputs = self.relu(outputs)
        outputs = self.linear2(table_sizes)
        outputs = self.relu(outputs)
        outputs = self.linear3(table_sizes)
        outputs = self.relu(outputs)
        outputs = self.linear4(table_sizes)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = Regressor()

# connect to gpu
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_labels, batch_table_sizes = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_table_sizes).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_labels, b_table_sizes = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_table_sizes)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_labels, b_table_sizes = batch

                eval_output =  model(b_table_sizes)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for visualisation
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else 1e-10) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else 1e-10) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

#Limited syntax analysis

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)


input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)
input_table_sizes = <put equivalent table sizes here>

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]
input_table_sizes = input_table_sizes#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries,
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

size = len(input_queries)

# Create segment embeddings
segments = [[0] * max_length]
# Initialize with zeroes
for i in range(size-1):
  segments += [[0] * max_length]
# For every query
for i, query in enumerate(input_queries):
    broken_query = query.replace(">=", " >= ")
    broken_query = broken_query.replace("<=", " <= ")
    broken_query = broken_query.replace("!=", " != ")
    broken_query = broken_query.replace(">", " > ")
    broken_query = broken_query.replace("<", " < ")
    broken_query = broken_query.replace("=", " = ")
    broken_query = broken_query.replace(",", " , ")
    j = 1
    state = 0
    # Label each token based on its basic syntactic purpose
    for word in broken_query.split(" "):
        # First for keywords
        if word == 'SELECT':
          state = 0
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 0
            j += 1
        elif word == 'FROM':
          state = 1
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 1
            j += 1
        elif word == 'WHERE':
          state = 2
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 2
            j += 1
        elif word == 'COUNT(*)':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 3
            j += 1
        elif word == 'AND':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 4
            j += 1
        elif word == 'UNION':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 5
            j += 1
        elif word == 'IN':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 6
            j += 1
        elif word == '=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 7
            j += 1
        elif word == '>=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 8
            j += 1
        elif word == '<=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 9
            j += 1
        elif word == '>':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 10
            j += 1
        elif word == '<':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 11
            j += 1
        elif word == ',':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 12
            j += 1
        # Then for other terms
        else:
          # Return columns and tables
          if state < 2:
            for k in range(len(tokenizer.tokenize(word))):
              segments[i][j] = 13 + state
              j += 1
          else:
            # Column in expression
            if "." in word:
              for k in range(len(tokenizer.tokenize(word))):
                segments[i][j] = 15
                j += 1
            # Value
            else:
              for k in range(len(tokenizer.tokenize(word))):
                segments[i][j] = 16
                j += 1
    segments[i][j] = segments[i][j-1]

# function changing embeddings into binary so that they are acceptable by BERT
def change_to_binary(lst):
    binary_lst = []
    for i in range(len(lst)):
        if i == 0:
            binary_lst.append(0)
        elif lst[i] != lst[i-1]:
            binary_lst.append(1-binary_lst[i-1])
        else:
            binary_lst.append(binary_lst[i-1])
    return binary_lst

segments = [change_to_binary(segment) for segment in segments]

# spliting data into train, validation and test sets

input_ids_test = input_ids[int(size*0.8):]
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_test = attention_mask[int(size*0.8):]
attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

segments_test = segments[int(size*0.8):]
segments_val = segments[int(size*0.6):int(size*0.8)]
segments_train = segments[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

table_sizes_test = input_table_sizes[int(size*0.8):]
table_sizes_val = input_table_sizes[int(size*0.6):int(size*0.8)]
table_sizes_train = input_table_sizes[:int(size*0.6)]

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize_simple(self, train, validation, test):
        train_stand = torch.tensor(train, dtype=torch.float)
        validation_stand = torch.tensor(validation, dtype=torch.float)
        test_stand = torch.tensor(test, dtype=torch.float)

        stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / stds
        validation_stand = validation_stand / stds
        test_stand = test_stand / stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)
table_sizes_train, table_sizes_val, table_sizes_test = standardizer.standardize_simple(table_sizes_train, table_sizes_val, table_sizes_test)

from torch.utils.data import TensorDataset, DataLoader

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, segments, labels, table_sizes, batch_size):
    labels_tens = torch.tensor(labels)
    segments_tens = torch.tensor(segments)
    table_sizes_tens = torch.tensor(table_sizes, dtype=torch.float)
    dataset = TensorDataset(inputs, masks, segments_tens, labels_tens, table_sizes_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, segments_train, labels_train, table_sizes_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, segments_val, labels_val, table_sizes_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, segments_test, labels_test, table_sizes_test, batch_size)

labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

import torch.nn as nn
from transformers import BertForSequenceClassification, BertConfig, BertModel

# load bert model within our defined network
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in+4, D_out)  #+4 because we are adding table sizes
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks, segments, table_sizes):
        outputs = self.bert(input_ids, attention_mask=attention_masks, token_type_ids=segments)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        embedding_with_sizes = torch.cat(tensors=[outputs, table_sizes], dim=1)
        outputs = self.linear(embedding_with_sizes)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to gpu
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_segments, batch_labels, batch_table_sizes = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks, batch_segments, batch_table_sizes).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_segment, b_labels, b_table_sizes = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask, b_segment, b_table_sizes)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_segment, b_labels, b_table_sizes = batch

                eval_output =  model(b_input_ids, b_input_mask, b_segment, b_table_sizes)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for visualisation
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else 1e-10) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else 1e-10) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

#L − 8H − 256A − 4

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

# obtaining input queries and labels (cardinality or cost)
input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
size = len(input_queries)

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-8_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries[:int(size*0.8)],
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

encoded_corpus_test = tokenizer(text=input_queries[int(size*0.8):],
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids_test = encoded_corpus_test['input_ids']
attention_mask_test = encoded_corpus_test['attention_mask']

# split train, validation and test sets
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize(self, train, validation, test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test = standardizer.standardize(labels_train, labels_val, labels_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, batch_size):
    labels_tens = torch.tensor(labels)
    dataset = TensorDataset(inputs, masks, labels_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, batch_size)


# destandardize labels after passing them to the dataloaders,
# we want them standardized within the model and unstandardized outside
labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)

class BertRegressor(nn.Module):
    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, D_out)
        self.relu = nn.ReLU()
    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                eval_output =  model(b_input_ids, b_input_mask)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

#SyntaxBERT

In [ ]:
!pip install fastdtw

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel
from fastdtw import fastdtw
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

# obtaining input queries and labels (cardinality or cost)
input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
size = len(input_queries)

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries[:int(size*0.8)],
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

encoded_corpus_test = tokenizer(text=input_queries[int(size*0.8):],
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids_test = encoded_corpus_test['input_ids']
attention_mask_test = encoded_corpus_test['attention_mask']

# split train, validation and test sets
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# Create segment embeddings
segments = [[0] * max_length]
# Initialize with zeroes
for i in range(size-1):
    segments += [[0] * max_length]
# For every query
for i, query in enumerate(input_queries):
    broken_query = query.replace(">=", " >= ")
    broken_query = broken_query.replace("<=", " <= ")
    broken_query = broken_query.replace("!=", " != ")
    broken_query = broken_query.replace(">", " > ")
    broken_query = broken_query.replace("<", " < ")
    broken_query = broken_query.replace("=", " = ")
    broken_query = broken_query.replace(",", " , ")
    j = 1
    state = 0
    # Label each token based on its basic syntactic purpose
    for word in broken_query.split(" "):
        # First for keywords
        if word == 'SELECT':
          state = 0
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 0
            j += 1
        elif word == 'FROM':
          state = 1
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 1
            j += 1
        elif word == 'WHERE':
          state = 2
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 2
            j += 1
        elif word == 'COUNT(*)':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 3
            j += 1
        elif word == 'AND':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 4
            j += 1
        elif word == 'UNION':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 5
            j += 1
        elif word == 'IN':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 6
            j += 1
        elif word == '=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 7
            j += 1
        elif word == '>=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 8
            j += 1
        elif word == '<=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 9
            j += 1
        elif word == '>':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 10
            j += 1
        elif word == '<':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 11
            j += 1
        elif word == ',':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 12
            j += 1
        # Then for other terms
        else:
          # Return columns and tables
          if state < 2:
            for k in range(len(tokenizer.tokenize(word))):
              segments[i][j] = 13 + state
              j += 1
          else:
            # Column in expression
            if "." in word:
              for k in range(len(tokenizer.tokenize(word))):
                segments[i][j] = 15
                j += 1
            # Value
            else:
              for k in range(len(tokenizer.tokenize(word))):
                segments[i][j] = 16
                j += 1
    segments[i][j] = segments[i][j-1]
    segments[i] = segments[i][:j+1]

# split the segments into a train, a validation and a test set
segments_test = segments[int(size*0.8):]
segments_val = segments[int(size*0.6):int(size*0.8)]
segments_train = segments[:int(size*0.6)]

n = len(segments_train)
# Calculate DTW distances in the train set and create a distance matrix
distance_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(i, n):
        distance, _ = fastdtw(segments_train[i], segments_train[j])
        distance_matrix[i][j] = distance
        distance_matrix[j][i] = distance

# Perform K-Means clustering
num_clusters = 5  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(distance_matrix)

# Perform PCA for dimensionality reduction
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(distance_matrix)

# Visualize the cluster assignments as points on a 2D plane
plt.figure(figsize=(8, 6))
colors = ['r', 'g', 'b', 'm', 'y']  # Assign colors to clusters
for i in range(num_clusters):
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    plt.scatter(reduced_data[cluster_indices, 0], reduced_data[cluster_indices, 1], label=f'Cluster {i}', c=colors[i], s=50)

plt.xlabel('PCA Dimension 1')
plt.ylabel('PCA Dimension 2')
plt.title('K-Means Clustering Visualization of Time Series Data')
plt.legend()
plt.grid(True)
plt.show()

# Extract points of each cluster
cluster_points = [[] for _ in range(num_clusters)]
for i in range(len(segments_train)):
    cluster_index = kmeans.labels_[i]
    cluster_points[cluster_index].append(input_labels[i])

# Average the points of each cluster
centroid_labels = []
for points in cluster_points:
    cur_sum = 0
    for point in points:
        cur_sum += point
    average = cur_sum/len(points)
    centroid_labels.append(average)

# Get the cluster centroids
centroids = kmeans.cluster_centers_

# Match the train set to a clusters
predicted_from_clust_train = []
for i in range(len(segments_train)):
    centr_pos = np.argmin([fastdtw(segments_train[i], centroid)[0] for centroid in centroids])
    matching_centr_label = centroid_labels[centr_pos]
    predicted_from_clust_train.append(matching_centr_label)

# Match the val set to a clusters
predicted_from_clust_val = []
for i in range(len(segments_val)):
    centr_pos = np.argmin([fastdtw(segments_val[i], centroid)[0] for centroid in centroids])
    matching_centr_label = centroid_labels[centr_pos]
    predicted_from_clust_val.append(matching_centr_label)

# Match the test set to a clusters
predicted_from_clust_test = []
for i in range(len(segments_test)):
    centr_pos = np.argmin([fastdtw(segments_test[i], centroid)[0] for centroid in centroids])
    matching_centr_label = centroid_labels[centr_pos]
    predicted_from_clust_test.append(matching_centr_label)

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize(self, train, validation, test, clust_train, clust_val, clust_test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)
        clust_train_stand = torch.tensor(clust_train)
        clust_val_stand = torch.tensor(clust_val)
        clust_test_stand = torch.tensor(clust_test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds
        clust_train_stand = clust_train_stand / self.stds
        clust_val_stand = clust_val_stand / self.stds
        clust_test_stand = clust_test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        clust_train_stand = clust_train_stand.detach().cpu().tolist()
        clust_val_stand = clust_val_stand.detach().cpu().tolist()
        clust_test_stand = clust_test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand, clust_train_stand, clust_val_stand, clust_test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test, predicted_from_clust_train, predicted_from_clust_val, predicted_from_clust_test = standardizer.standardize(labels_train, labels_val, labels_test, predicted_from_clust_train, predicted_from_clust_val, predicted_from_clust_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, predicted_from_clust, batch_size):
    labels_tens = torch.tensor(labels)
    predicted_from_clust_tens = torch.tensor(predicted_from_clust)
    dataset = TensorDataset(inputs, masks, labels_tens, predicted_from_clust_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, predicted_from_clust_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, predicted_from_clust_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, predicted_from_clust_test, batch_size)

# destandardize labels after passing them to the dataloaders,
# we want them standardized within the model and unstandardized outside
labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)
predicted_from_clust_train = standardizer.destandardize(predicted_from_clust_train)
predicted_from_clust_val = standardizer.destandardize(predicted_from_clust_val)
predicted_from_clust_test = standardizer.destandardize(predicted_from_clust_test)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)


class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, 1)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(2, D_out)
        self.relu2 = nn.ReLU()
    def forward(self, input_ids, attention_masks, predicted_from_clust):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        outputs = torch.stack((outputs, predicted_from_clust),1)
        outputs = self.linear2(outputs)
        outputs = self.relu2(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels, batch_clust = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks, batch_clust).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels, b_clust = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask, b_clust)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels, b_clust = batch

                eval_output =  model(b_input_ids, b_input_mask, b_clust)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

# Truncation, testing clusters individually

In [ ]:
!pip install fastdtw

In [ ]:
# necessary imports
import re
import statistics
import torch
from transformers import AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import BertConfig, BertModel
from fastdtw import fastdtw
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# function transforming numerical values in our dataset into tokens representing ranges
def num_to_token(num):
  if num < 1850:
    return str(num)
  if num < 2000:
    return "year1"
  elif num < 2010:
    return "year2"
  else:
    return "year3"

# function performing basic preprocessing
def preprocess_query(query):
  return re.sub(r"(\d+)", lambda x: num_to_token(int(x.group(0))), query)

# obtaining input queries and labels (cardinality or cost)
input_queries = [preprocess_query(query) for query in <put desired query dataset here>]
input_labels = <put desired target here> #("labels" for JOB-light cardinality and "costs" for JOB-light costs)

# limiting the number of queries to be used in the model
input_queries = input_queries#[:number of queries to be used]
input_labels = input_labels#[:number of queries to be used]

# maximum query length before truncation, in number of tokens
max_length = 256

# tokenize dataset
size = len(input_queries)

# tokenize dataset
tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-4_H-256_A-4', use_fast=True)
encoded_corpus = tokenizer(text=input_queries,
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_length,
                            return_attention_mask=True,
                            return_tensors='pt')
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

# split train, validation and test sets
input_ids_val = input_ids[int(size*0.6):int(size*0.8)]
input_ids_train = input_ids[:int(size*0.6)]

attention_mask_val = attention_mask[int(size*0.6):int(size*0.8)]
attention_mask_train = attention_mask[:int(size*0.6)]

labels_test = input_labels[int(size*0.8):]
labels_val = input_labels[int(size*0.6):int(size*0.8)]
labels_train = input_labels[:int(size*0.6)]

# calculate the mean of the target in the train data
train_mean = statistics.mean(labels_train)

# Create segment embeddings
segments = [[0] * max_length]
# Initialize with zeroes
for i in range(size-1):
    segments += [[0] * max_length]
# For every query
for i, query in enumerate(input_queries):
    broken_query = query.replace(">=", " >= ")
    broken_query = broken_query.replace("<=", " <= ")
    broken_query = broken_query.replace("!=", " != ")
    broken_query = broken_query.replace(">", " > ")
    broken_query = broken_query.replace("<", " < ")
    broken_query = broken_query.replace("=", " = ")
    broken_query = broken_query.replace(",", " , ")
    j = 1
    state = 0
    # Label each token based on its basic syntactic purpose
    for word in broken_query.split(" "):
        # First for keywords
        if word == 'SELECT':
          state = 0
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 0
            j += 1
        elif word == 'FROM':
          state = 1
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 1
            j += 1
        elif word == 'WHERE':
          state = 2
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 2
            j += 1
        elif word == 'COUNT(*)':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 3
            j += 1
        elif word == 'AND':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 4
            j += 1
        elif word == 'UNION':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 5
            j += 1
        elif word == 'IN':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 6
            j += 1
        elif word == '=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 7
            j += 1
        elif word == '>=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 8
            j += 1
        elif word == '<=':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 9
            j += 1
        elif word == '>':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 10
            j += 1
        elif word == '<':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 11
            j += 1
        elif word == ',':
          for k in range(len(tokenizer.tokenize(word))):
            segments[i][j] = 12
            j += 1
        # Then for other terms
        else:
          # Return columns and tables
          if state < 2:
            for k in range(len(tokenizer.tokenize(word))):
              segments[i][j] = 13 + state
              j += 1
          else:
            # Column in expression
            if "." in word:
              for k in range(len(tokenizer.tokenize(word))):
                segments[i][j] = 15
                j += 1
            # Value
            else:
              for k in range(len(tokenizer.tokenize(word))):
                segments[i][j] = 16
                j += 1
    segments[i][j] = segments[i][j-1]
    segments[i] = segments[i][:j+1]

# Calculate DTW distances in the train set and create a distance matrix
n = len(segments)
distance_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(i, n):
        distance, _ = fastdtw(segments[i], segments[j])
        distance_matrix[i][j] = distance
        distance_matrix[j][i] = distance

# Perform K-Means clustering
num_clusters = 70  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(distance_matrix)

# Extract points of each cluster
cluster_points = [[] for _ in range(num_clusters)]
for i in range(len(segments)):
    cluster_index = kmeans.labels_[i]
    cluster_points[cluster_index].append((input_ids[i], attention_mask[i],input_labels[i], segments[i]))

# create test sets containing some queries from each cluster
test_sets = [[] for _ in range(70)]
for i in range(70):
    test_sets[i].append(cluster_points[i].pop())
    if len(cluster_points[i]) > 5:
        test_sets[i] += cluster_points[i][4*len(cluster_points[i])//5:]
        cluster_points[i] = cluster_points[i][:4*len(cluster_points[i])//5]
import itertools
test_set = list(itertools.chain.from_iterable(test_sets))

input_ids_tests = [[] for _ in range(70)]
for i in range(70):
    input_ids_tests[i] = [j[0] for j in test_sets[i]]
attention_mask_tests = [[] for _ in range(70)]
for i in range(70):
    attention_mask_tests[i] = [j[1] for j in test_sets[i]]
labels_tests = [[] for _ in range(70)]
for i in range(70):
    labels_tests[i] = [j[2] for j in test_sets[i]]
segments_tests = [[] for _ in range(70)]
for i in range(70):
    segments_tests[i] = [j[3] for j in test_sets[i]]
input_ids_test = list(itertools.chain.from_iterable(input_ids_tests))
attention_mask_test = list(itertools.chain.from_iterable(attention_mask_tests))
labels_test = list(itertools.chain.from_iterable(labels_tests))
segments_test = list(itertools.chain.from_iterable(segments_tests))
predicted_from_clust_tests = labels_tests

# create train and validation sets from the remaining data
all_remaining = list(itertools.chain.from_iterable(cluster_points))
train_all = all_remaining[:int(size*0.75)]
input_ids_train = [j[0] for j in train_all]
attention_mask_train = [j[1] for j in train_all]
labels_train = [j[2] for j in train_all]
segments_train = [j[3] for j in train_all]
val_all = all_remaining[int(size*0.75):]
input_ids_val = [j[0] for j in train_all]
attention_mask_val = [j[1] for j in train_all]
labels_val = [j[2] for j in train_all]
segments_val = [j[3] for j in train_all]

# Average the points of each cluster
centroid_labels = []
for points in cluster_points:
    cur_sum = 0
    for point in points:
        cur_sum += 5
    average = cur_sum/5
    centroid_labels.append(average)

# Get the cluster centroids
centroids = kmeans.cluster_centers_

# Match the train set to a clusters
predicted_from_clust_train = []
for i in range(len(segments_train)):
    matching_centr_label = centroid_labels[0]
    predicted_from_clust_train.append(matching_centr_label)

# Match the val set to a clusters
predicted_from_clust_val = []
for i in range(len(segments_val)):
    matching_centr_label = centroid_labels[0]
    predicted_from_clust_val.append(matching_centr_label)

# Match the test set to a cluster
predicted_from_clust_test = []
for i in range(len(segments_test)):
    matching_centr_label = centroid_labels[0]
    predicted_from_clust_test.append(matching_centr_label)

# standardize all data, based on the statistics of the train set
class Standardizer:
    def __init__(self):
        self.means = 0
        self.stds = 0

    def standardize(self, train, validation, test, clust_train, clust_val, clust_test):
        train_stand = torch.tensor(train)
        validation_stand = torch.tensor(validation)
        test_stand = torch.tensor(test)
        clust_train_stand = torch.tensor(clust_train)
        clust_val_stand = torch.tensor(clust_val)
        clust_test_stand = torch.tensor(clust_test)

        self.stds = train_stand.std(dim=0, keepdim=True)
        train_stand = train_stand / self.stds
        validation_stand = validation_stand / self.stds
        test_stand = test_stand / self.stds
        clust_train_stand = clust_train_stand / self.stds
        clust_val_stand = clust_val_stand / self.stds
        clust_test_stand = clust_test_stand / self.stds

        train_stand = train_stand.detach().cpu().tolist()
        validation_stand = validation_stand.detach().cpu().tolist()
        test_stand = test_stand.detach().cpu().tolist()
        clust_train_stand = clust_train_stand.detach().cpu().tolist()
        clust_val_stand = clust_val_stand.detach().cpu().tolist()
        clust_test_stand = clust_test_stand.detach().cpu().tolist()
        return train_stand, validation_stand, test_stand, clust_train_stand, clust_val_stand, clust_test_stand

    def destandardize(self, data):
        data_destand = torch.tensor(data)
        data_destand = data_destand*self.stds
        data_destand = data_destand.detach().cpu().tolist()
        return data_destand

standardizer = Standardizer()
labels_train, labels_val, labels_test, predicted_from_clust_train, predicted_from_clust_val, predicted_from_clust_test = standardizer.standardize(labels_train, labels_val, labels_test, predicted_from_clust_train, predicted_from_clust_val, predicted_from_clust_test)

# make dataloaders
batch_size = 64

def make_dataloader(inputs, masks, labels, predicted_from_clust, batch_size):
    predicted_from_clust = [0]*len(labels)
    inputs_tens = torch.stack(inputs)
    masks_tens = torch.stack(masks)
    labels_tens = torch.tensor(labels)
    predicted_from_clust_tens = torch.tensor(predicted_from_clust)
    dataset = TensorDataset(inputs_tens, masks_tens, labels_tens, predicted_from_clust_tens)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

train_dataloader = make_dataloader(input_ids_train, attention_mask_train, labels_train, predicted_from_clust_train, batch_size)
valid_dataloader = make_dataloader(input_ids_val, attention_mask_val, labels_val, predicted_from_clust_val, batch_size)
test_dataloader = make_dataloader(input_ids_test, attention_mask_test, labels_test, predicted_from_clust_test, batch_size)
test_dataloaders = []
for i in range(70):
  test_dataloaders.append(make_dataloader(input_ids_tests[i],attention_mask_tests[i],labels_tests[i],predicted_from_clust_tests[i], batch_size))

# destandardize labels after passing them to the dataloaders,
# we want them standardized within the model and unstandardized outside
labels_train = standardizer.destandardize(labels_train)
labels_val = standardizer.destandardize(labels_val)
labels_test = standardizer.destandardize(labels_test)
predicted_from_clust_train = standardizer.destandardize(predicted_from_clust_train)
predicted_from_clust_val = standardizer.destandardize(predicted_from_clust_val)
predicted_from_clust_test = standardizer.destandardize(predicted_from_clust_test)

# load BERT model and define our head
config = BertConfig.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2, hidden_dropout_prob=0.3,
                                    attention_probs_dropout_prob=0.3, output_attentions = False, output_hidden_states = True)


class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.3, config=config):

        super(BertRegressor, self).__init__()
        D_in, D_out = 256, 1

        self.bert = BertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', config=config)
        self.drop = nn.Dropout(drop_rate)
        self.linear = nn.Linear(D_in, 1)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(2, D_out)
        self.relu2 = nn.ReLU()
    def forward(self, input_ids, attention_masks, predicted_from_clust):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][:]
        token_vecs = token_vecs.permute(1,0,2)
        sentence_embedding = torch.mean(token_vecs, dim=0)
        outputs = self.drop(sentence_embedding)
        outputs = self.linear(outputs)
        outputs = self.relu(outputs)
        outputs = torch.squeeze(outputs, 1)
        outputs = torch.stack((outputs, predicted_from_clust),1)
        outputs = self.linear2(outputs)
        outputs = self.relu2(outputs)
        outputs = torch.squeeze(outputs, 1)
        return outputs

model = BertRegressor()

# connect to GPU, if available
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define optimizer, with scheduler for adapting learning rate
from transformers import get_linear_schedule_with_warmup

epochs = 50 # define maximum number of epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# define loss function
loss_function = nn.MSELoss()

# imports for statistics
from sklearn.metrics import r2_score
import statistics

# define statistics
list_r2=[]
list_r2_train=[]
list_qerror=[]
list_qerror_train=[]
list_sample_size=[]

# function to predict using model
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, batch_labels, batch_clust = tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, batch_masks, batch_clust).view(1,-1).tolist()[0]
    return output

# function to train model
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, valid_dataloader, device, clip_value=2):
    best_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        # print progress
        print(f"Epoch {epoch+1}/{epochs}")
        print("-----")
        model.train()
        train_loss = 0.0

        y_train = []
        print("Training:")
        for step, batch in enumerate(train_dataloader):
            if step == 0:
              print("1/"+str(len(train_dataloader)), end='')
            else:
              print('\b'*(len(str(step)) + len(str(len(train_dataloader))) + 1) + str(step + 1) + "/" + str(len(train_dataloader)), end='')

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels, b_clust = batch

            optimizer.zero_grad()
            # Forward pass
            predictions = model(b_input_ids, b_input_mask, b_clust)
            y_train += predictions.detach().cpu().tolist()
            loss = loss_function(predictions, b_labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            # Update model's weights based on the gradients calculated during backpropagation
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        print('\b'*(2*len(str(len(train_dataloader)))+12))

        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Training Loss: {avg_train_loss}")

        # Evaluate on validation set
        model.eval()
        eval_loss = 0.0

        y_val = []
        with torch.no_grad():
            print("Validating:")
            for step, batch in enumerate(valid_dataloader):
                if step == 0:
                  print("1/"+str(len(valid_dataloader)), end='')
                else:
                  print('\b'*(len(str(step)) + len(str(len(valid_dataloader))) + 1) + str(step + 1) + "/" + str(len(valid_dataloader)), end='')

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels, b_clust = batch

                eval_output =  model(b_input_ids, b_input_mask, b_clust)
                y_val += eval_output.detach().cpu().tolist()
                loss = loss_function(eval_output, b_labels)

                eval_loss += loss

        print('\b'*(2*len(str(len(train_dataloader)))+14))

        avg_eval_loss = eval_loss / len(valid_dataloader)
        print(f"Validation Loss: {avg_eval_loss}")

        # calculate extra statistics for the output
        y_train = standardizer.destandardize(y_train)
        y_val = standardizer.destandardize(y_val)

        r2_train = r2_score(labels_train, y_train)
        print("R² Train: " + str(r2_train))
        r2 = r2_score(labels_val, y_val)
        print("R² Validation: " + str(r2))

        qerror_train = statistics.mean([max(labels_train[i], y_train[i])/min(labels_train[i], y_train[i] if y_train[i] > 0 else train_mean) for i in range(len(labels_train))])
        print("Q-error Train: " + str(qerror_train))
        qerror = statistics.mean([max(labels_val[i], y_val[i])/min(labels_val[i], y_val[i] if y_val[i] > 0 else train_mean) for i in range(len(labels_val))])
        print("Q-error Validation: " + str(qerror))
        print()

        list_r2.append(r2)
        list_r2_train.append(r2_train)
        list_qerror.append(qerror)
        list_qerror_train.append(qerror_train)
        list_sample_size.append((epoch * 0.1 + 0.1))

        # Early stopping based on validation loss
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == 5:
                print("Validation loss has not improved in 5 epochs, stopping early.")
                break

    return model

# Train model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, valid_dataloader, device, clip_value=2)
# Testing

# define r2_score function
def r2_score(outputs, labels):
    labels_mean = statistics.mean(labels)
    ss_tot = sum([(x-labels_mean)**2 for x in labels])
    ss_res = sum([(x-y)**2 for x,y in zip(labels, outputs)])
    r2 = (1 - (ss_res / (ss_tot if ss_tot != 0 else 1e-10)))
    return r2

# Measure performance on test set
y_test = labels_test
y_pred = predict(model, test_dataloader, device)
y_pred = standardizer.destandardize(y_pred)

# more imports for metrics to be used on test set
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pandas as pd

mae = mean_absolute_error(y_test, y_pred)
print("MAE Test: ", mae)
mdae = median_absolute_error(y_test, y_pred)
print("MDAE Test: ", mdae)
mse = mean_squared_error(y_test, y_pred)
print("MSE Test: ", mse)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE Test: ", mape)
mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
print("MDAPE Test: ", mdape)
r_squared = r2_score(y_test, y_pred)
print("R² Test: ", r_squared)
qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
print("Q-error Test: ", qerror)

qerrors = []
# Measure performance on each of the 70 test sets
for i in range(70):
  print(i)
  y_test = labels_tests[i]
  y_pred = predict(model, test_dataloaders[i], device)
  y_pred = standardizer.destandardize(y_pred)

  mae = mean_absolute_error(y_test, y_pred)
  print("MAE Test: ", mae)
  mdae = median_absolute_error(y_test, y_pred)
  print("MDAE Test: ", mdae)
  mse = mean_squared_error(y_test, y_pred)
  print("MSE Test: ", mse)
  mape = mean_absolute_percentage_error(y_test, y_pred)
  print("MAPE Test: ", mape)
  mdape = ((pd.Series(y_test) - pd.Series(y_pred)) / pd.Series(y_test)).abs().median()
  print("MDAPE Test: ", mdape)
  r_squared = r2_score(y_test, y_pred)
  print("R² Test: ", r_squared)
  qerror = statistics.mean([max(y_test[i], y_pred[i] if y_pred[i] != 0 else train_mean)/min(y_test[i], y_pred[i] if y_pred[i] > 0 else train_mean) for i in range(len(y_test))])
  print("Q-error Test: ", qerror)
  qerrors.append(qerror)

  for j in range(len(y_test)):
    mae = mean_absolute_error([y_test[j]], [y_pred[j]])
    #print("MAE Test: ", mae)
    mdae = median_absolute_error([y_test[j]], [y_pred[j]])
    #print("MDAE Test: ", mdae)
    mse = mean_squared_error([y_test[j]], [y_pred[j]])
    #print("MSE Test: ", mse)
    mape = mean_absolute_percentage_error([y_test[j]], [y_pred[j]])
    #print("MAPE Test: ", mape)
    mdape = ((pd.Series([y_test[j]]) - pd.Series([y_pred[j]])) / pd.Series([y_test[j]])).abs().median()
    #print("MDAPE Test: ", mdape)
    r_squared = r2_score([y_test[j]], [y_pred[j]])
    print("R² Test: ", r_squared)
    qerror = max(y_test[j], y_pred[j] if y_pred[j] != 0 else train_mean)/min(y_test[j], y_pred[j] if y_pred[j] > 0 else train_mean)
    print("Q-error Test: ", qerror)

# Your list of numbers
categories = [" "*i for i in range(70)]

# Create a bar chart
plt.bar(categories, qerrors)

# Add labels and a title
plt.xlabel('Sets')
plt.ylabel('q-error')
plt.title('q-error metric for various test subsets')

# Display the chart
plt.show()
